# Text AutoAugment Demo

In this notebook, we will conduct 2 demos about Text AutoAugment:

- **Use pre-search policy**: Use IMDB pre-search policy for augmenting English examples.
- **Search policy for Vietnamese dataset**: Use Vietnamese dataset for search policy and then augment some Vietnamese examples.

## Demo 1: Use pre-searched policy

### Read configuration

In [1]:
from theconf import Config as C

configfile="./taa/confs/english_data.yaml"
C(configfile)

### Load custom dataset

In [2]:
from datasets import load_dataset # load huggingface dataset

train_dataset = load_dataset(path="./taa/data", data_files={'train': 'english_data_train.csv', 'test': 'custom_data_test.csv'}, split='train')
train_dataset.data

/home/tuan/anaconda3/envs/taa/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration data-696021b8a1128382
Found cached dataset csv (/home/tuan/.cache/huggingface/datasets/csv/data-696021b8a1128382/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


MemoryMappedTable
text: string
label: int64
----
text: [["The sailors rode the breeze clear of the rocks.","The more you would want, the less you would eat."]]
label: [[1,0]]

### Train TF-IDF models for replacements 

In [3]:
from taa.utils.train_tfidf import train_tfidf

train_tfidf("imdb")  # calculate tf-idf score for TS and TI operations

Use exist TF-IDF model


### Initialize pre-search policy from IMDB dataset

In [4]:
from taa.archive import policy_map

policy = policy_map["imdb"]
policy

[[('tfidf_word_substitute', 0.7701989738031709, 0.1413189624238933),
  ('tfidf_word_substitute', 0.5017627182356647, 0.6120126108801994)],
 [('random_word_delete', 0.723773044425342, 0.09343851538354664),
  ('tfidf_word_insert', 0.65871023661296, 0.18515171972877364)],
 [('tfidf_word_insert', 0.6855258510775246, 0.25155717455902515),
  ('random_word_swap', 0.5855398958576279, 0.8038331622764382)],
 [('tfidf_word_substitute', 0.766621630113518, 0.9996972209388315),
  ('tfidf_word_substitute', 0.35757492431715654, 0.5094646924565402)],
 [('tfidf_word_insert', 0.7201447184878109, 0.27394476379211474),
  ('tfidf_word_insert', 0.7102010737993143, 0.7735198668033543)],
 [('random_word_swap', 0.2602481801512967, 0.1094817565926513),
  ('random_word_swap', 0.377657390084737, 0.5860711221450985)],
 [('random_word_swap', 0.3628204769855412, 0.4174830136614628),
  ('tfidf_word_substitute', 0.7449752005673833, 0.5806015818156878)],
 [('random_word_delete', 0.6869314076454281, 0.30212959898754665),

In [5]:
from taa.transforms import Compose
from taa.data import Augmentation

transform_train = Compose([])
transform_train.transforms.insert(0, Augmentation(policy))

### Augment dataset

In [6]:
from transformers import BertTokenizer, BertTokenizerFast
from taa.utils.raw_data_utils import get_examples
from taa.custom_dataset import general_dataset

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# normalize text and get example in train dataset
examples = get_examples(train_dataset)

# augment dataset
augmented_dataset = general_dataset(examples, tokenizer, text_transform=transform_train)


### Get results

In [7]:
augmented_dataset.texts

['Sailors the rode the breeze sure clear of the. rocks',
 'The sailors rode the clear breeze the of. rocks',
 'The sailors rode the breeze of the rocks.',
 'Drowning the planned sailors rode the breeze presented clear of decide the rocks.',
 'More you the would want, less the would eat you.',
 'The more you would require, the le you would eat.',
 'More give the you would sure want the, less would you eat.',
 'The recursive delicatessen whining lousy, the less under earth eat.']

In [8]:
augmented_dataset.labels

[1, 1, 1, 1, 0, 0, 0, 0]

## Demo 2: Search policies for Vietnamese dataset

### The Vietnamese dataset

We crawled the articles and their categories from Thanhnien and Tuoitre newspaper.

We only take 5000 samples of articles, not the whole, for this demo.

First, we load the dataset:

In [1]:
import pandas as pd

df = pd.read_csv("./taa/data/data_lite.csv")
df.head(1)

,id,link,title,thumbnail,time,summary,category,sub_category,full_article,text
0,/post-1512613.html,https://thanhnien.vn/vinh-danh-va-ho-tro-lao-d...,Vinh danh và hỗ trợ lao động nữ phi chính thức...,https://image.thanhnien.vn/1200x630/Uploaded/2...,2022-10-20T16:30:43+0700,"Nhân ngày Phụ nữ Việt Nam 20.10 sắp đến, Unile...",Đời sống,Cộng đồng,"Nhân ngày Phụ nữ Việt Nam 20.10 sắp đến, Unil...",Vinh danh và hỗ trợ lao động nữ phi chính thức...


Transfrom into the form of TAA Dataset

In [2]:
df2 = pd.DataFrame()
df2["text"] = df["text"]
df2["label"] = df["category"]
df2

,text,label
0,Vinh danh và hỗ trợ lao động nữ phi chính thức...,Đời sống
1,"Chuyện lạ ở bản Robinson - Kỳ cuối: Phá đá, gi...",Thời sự
2,TNGT chết người do uống rượu bia: CSGT TP.HCM ...,Đời sống
3,Truyền thông Nhật chỉ ra mưu đồ của Trung Quốc...,Thế giới
4,Quyền Linh ngỡ ngàng trước người phụ nữ U.60 x...,Giải trí
...,...,...
4995,Hết tình ruột thịt. TT - Phe nguyên đơn tố bị ...,Pháp luật
4996,Hoa cúc dại và một vài khía cạnh phận người. T...,Văn hóa
4997,Con trai NSƯT Kim Phương tiết lộ lý do từng gi...,Giải trí
4998,Ba oanh tạc cơ Trung Quốc mang tên lửa hành tr...,Thế giới


In [5]:
df_train = pd.DataFrame(columns=["text", "label"])
df_test = pd.DataFrame(columns=["text", "label"])

categories = df2["label"].unique()
for i in range(len(categories)):
    dff = df2[df2["label"] == categories[i]].copy()
    dff["label"] = i
    df_train = pd.concat([df_train, dff.sample(50)])
    df_test = pd.concat([df_test, dff.sample(10)])


In [6]:
df_train.to_csv("./taa/data/vietnamese_data_train.csv", index=False)

In [7]:
df_test.to_csv("./taa/data/vietnamese_data_test.csv", index=False)

### Load config for training

In [1]:
from theconf import Config as C

configfile="./taa/confs/vietnamese_data.yaml"
C(configfile)

name = C.get()['dataset']['name']
path = C.get()['dataset']['path']
data_dir = C.get()['dataset']['data_dir']
data_files = C.get()['dataset']['data_files']
abspath = C.get()['abspath']

In [2]:
from taa.search import search_policy

policy = search_policy(dataset=name, abspath=abspath)

/home/tuan/anaconda3/envs/taa/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2022-12-05 02:17:18,177] [Text AutoAugment] [INFO] ----- Search Test-Time Augmentation Policies -----
INFO:Text AutoAugment:----- Search Test-Time Augmentation Policies -----
[2022-12-05 02:17:18,178] [Text AutoAugment] [INFO] ---------------------------------------------------
INFO:Text AutoAugment:---------------------------------------------------
[2022-12-05 02:17:18,179] [Text AutoAugment] [INFO] Loading configuration...
INFO:Text AutoAugment:Loading configuration...
[2022-12-05 02:17:18,180] [Text AutoAugment] [INFO] Initialize ray...
INFO:Text AutoAugment:Initialize ray...
2022-12-05 02:17:19,805	INFO worker.py:1528 -- Started a local Ray instance.
[2022-12-05 02:17:20,620] [Text AutoAugment] [INFO] Total computation fo

Make TF-IDF model directory: /mnt/d/Projects/text-autoaugment-demo/model_vietnamese_data/models/tfidf/vietnamese_data
Start training TF-IDF model. It will take a long time if the training dataset is too large


In [3]:
policy

[[('random_word_delete', 0.15261443795599705, 0.3110128094476705),
  ('tfidf_word_insert', 0.22059275510321485, 0.4649998836289846)],
 [('random_word_delete', 0.8058680196213712, 0.20768275560245286),
  ('tfidf_word_substitute', 0.4394515917253494, 0.4136585244395893)],
 [('tfidf_word_insert', 0.5845974078768156, 0.6622019273601056),
  ('random_word_swap', 0.527412040516822, 0.5540955328641466)],
 [('tfidf_word_substitute', 0.1587482939759266, 0.3513911551555692),
  ('tfidf_word_insert', 0.05255856587984242, 0.6633878446874389)]]

In [4]:
from datasets import load_dataset # load huggingface dataset

test_dataset = load_dataset(path="./taa/data", data_files={'train': 'vietnamese_data_train.csv', 'test': 'vietnamese_data_test.csv'}, split='test')

In [5]:
from taa.data import augment

augmented_dataset = augment(dataset=test_dataset, policy=policy, n_aug=4)

loading file vocab.txt from cache at /home/tuan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file tokenizer.json from cache at /home/tuan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/tuan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /home/tuan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing":

In [10]:
test_dataset.data["text"][0]

<pyarrow.StringScalar: 'Vợ chồng viên chức tỉnh thu nhập 10 triệu, nuôi 2 con nhỏ chi hết 15 triệu/tháng. Để đủ tiền chi tiêu cho một tháng, công nhân - viên chức sống tại TP.Đồng Hới Quảng Bình vẫn phải làm thêm mới đủ tiền chi tiêu trong gia đình; công nhân Quảng Trị dù trốn về quê làm việc vẫn xoay sở khó khăn trước bão giá.'>

In [7]:
augmented_dataset.texts[0:4]

['Độ vợ rộ chồng viên chức truyện tỉnh thu nhập 10 xẻ triệu, yếu nuôi Huy 2 con nhỏ chi hết bận 15 triệu / tháng. gánh Để đoan đủ tiền chi tiêu cho một 113 tháng, công vạ nhân - viên chức Nhái sống tại TP. Đồng Hới Quảng Sơn Bình vẫn phải khoắn làm thêm pháp mới khả đủ tiền SAIGONBANK chi tiêu trong chừng gia kiếp đình; chỉ công GDI nhân Quảng Trị khống dù trốn về quê làm quà việc vẫn buổi xoay sở khó khăn trước Thịnh bão giá.',
 'Vợ chồng viên chức tỉnh thu nhập 10 triệu, nuôi 2 con nhỏ chi hết 15 triệu/tháng. Để đủ tiền chi tiêu cho một tháng, công nhân - viên chức sống tại TP.Đồng Hới Quảng Bình vẫn phải làm thêm mới đủ tiền chi tiêu trong gia đình; công nhân Quảng Trị dù trốn về quê làm việc vẫn xoay sở khó khăn trước bão giá.',
 'Vợ chồng thâm viên lao chức tỉnh rốn thu nhập Alone 10 triệu, chở nuôi 2 khảnh con nhỏ vằn chi Cristian hết 15 triệu / tháng. thỉ Để đủ tiền Cẩm chi tiêu khôi cho Tuất một tháng, công thép nhân - viên giày chức sống tại TP. Tập Đồng Hới Quảng lỡ Bình vẫn 